# Setting up the enviroment

In [ ]:
# Cloning the git repo with the data structure
!git clone https://github.com/JessyD/test.git

In [ ]:
# Install necessary python dependencies
! pip install -r test/requirements.txt

# Download the Data

In [ ]:
!wget -O test/data/nspn.fmri.main.RData https://ndownloader.figshare.com/files/20958708

In [ ]:
!wget -O test/data/nspn.fmri.gsr.RData https://ndownloader.figshare.com/files/20958699

In [ ]:
!wget -O test/data/nspn.fmri.lowmot.RData https://ndownloader.figshare.com/files/20958702

In [ ]:
!wget -O test/data/nspn.fmri.general.vars.RData https://ndownloader.figshare.com/files/20819796

# Analysis

In [ ]:
import pyreadr 
import numpy as np
import matplotlib.pyplot as plt
import pylab
import matplotlib.colorbar
import bct
import pickle
from sklearn_rvm import EMRVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import plotly.express as px
from numpy.random import default_rng

from scipy import stats
from numpy.random import RandomState
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Set the random seed
np.random.seed(2)
rng = default_rng()

In [ ]:
# Define paths
from pathlib import Path
PROJECT_ROOT = Path.cwd()
data_path = PROJECT_ROOT / 'test' /'data'
output_path = PROJECT_ROOT / 'test' / 'output'

In [ ]:
data1 = pyreadr.read_r(str(data_path / 'nspn.fmri.main.RData'))
#genVar = pyreadr.read_r(str(data_path / 'nspn.fmri.general.vars.RData'))
#data2 = pyreadr.read_r(str(data_path / 'nspn.fmri.gsr.RData'))
#data3 = pyreadr.read_r(str(data_path / 'nspn.fmri.lowmot.RData'))

DataNames=['nspn.fmri.main.RData','nspn.fmri.gsr.RData','nspn.fmri.lowmot.RData']

In [ ]:
#Get motion regression functional connectivity data and reshape into region x region x subject array
FC=np.asarray(data1['fc.main'])
MainNoNan = np.nan_to_num(FC,copy=True,nan=1.0)
MainNoNanReshape=np.reshape(MainNoNan,[346,346,520],order='F')

#Get global signal regression functional connectivity data and reshape into region x region x subject arra

FC=np.asarray(data2['fc.gsr'])
GSRNoNan = np.nan_to_num(FC,copy=True,nan=1.0)
GSRNoNanReshape=np.reshape(GSRNoNan,[346,346,520],order='F')

#Read in subject IDs and age
IDMain=np.asarray(data1['id.main'])
Ages=np.asarray(data1['age.main'])

#Find unique subject IDs and index of first instance and find FC data corresponding to these indices
IDs,IDIndexUnique=np.unique(IDMain,return_index=True)
MainNoNanReshapeUnique=MainNoNanReshape[:,:,IDIndexUnique]
GSRNoNanReshapeUnique=GSRNoNanReshape[:,:,IDIndexUnique]
AgesUnique=Ages[IDIndexUnique]

#Number of randomly selected subjects to be used to define the low-dimensional space then split FC data and age data into two: 50 for defining space and remaining 248 for subsequent prediction
SpaceDefineN=50
RandomIndexes = rng.choice(IDs.shape[0], size=IDs.shape[0], replace=False)
MainNoNanModelSpace=MainNoNanReshapeUnique[:,:,RandomIndexes[0:SpaceDefineN]]
MainNoNanPrediction=MainNoNanReshapeUnique[:,:,RandomIndexes[SpaceDefineN:]]
GSRNoNanModelSpace=GSRNoNanReshapeUnique[:,:,RandomIndexes[0:SpaceDefineN]]
GSRNoNanPrediction=GSRNoNanReshapeUnique[:,:,RandomIndexes[SpaceDefineN:]]
AgesModelSpace=AgesUnique[RandomIndexes[0:SpaceDefineN]]
AgesPrediction=AgesUnique[RandomIndexes[SpaceDefineN:]]
IDsModelSpace=IDs[RandomIndexes[0:SpaceDefineN]] 
IDsPrediction=IDs[RandomIndexes[SpaceDefineN:]]


In [ ]:
#Get info about brain regions and find Yeo network IDs; useful later on for graph metrics that need community labels.
KeptIDs=np.asarray(genVar['hcp.keep.id'])
YeoIDs=np.asarray(genVar['yeo.id.subc'])
KeptYeoIDs=YeoIDs[KeptIDs-1][:,0,0]

In [ ]:
#Dictionary of 16 graph theory measures taken from the Brain Connectivity Toolbox

BCT_models = [
    ('00_Undirected_degree', bct.degrees_und),
    ('01_Undirected_strength', bct.strengths_und),
    ('02_Betweeness_Bin', bct.betweenness_bin),
    ('03_Clust_Bu', bct.clustering_coef_bu),
    ('04_Clust_Wu', bct.clustering_coef_wu),
    ('05_EigenvectorC', bct.eigenvector_centrality_und),
    ('06_SubgraphC', bct.subgraph_centrality),
    ('07_LocalEfficiencyC', bct.efficiency_bin),
    ('08_ModularityLouvain', bct.modularity_louvain_und),
    ('09_ModularityProbTune', bct.modularity_probtune_und_sign),
    ('10_ParticipationCoef', bct.participation_coef),
    ('11_ModuleDegreeZScore', bct.module_degree_zscore),
    ('12_PagerankCentrality', bct.pagerank_centrality),
    ('13_DiversityCoef', bct.diversity_coef_sign),
    ('14_GatewayDegree',bct.gateway_coef_sign),
    ('15_KCoreCentrality',bct.kcoreness_centrality_bu),
]

## Generating data to build low-dimensional space

In [ ]:
#This involves exhaustive evaluation of all 544 analysis approaches.  
TotalRegions=346
Results={}
BCT_Run={}
Sparsities_Run={}
Data_Run={}
GroupSummary={}
Results=np.array([])
ResultsIndVar=np.array([])

count=0
for DataPreproc in range(0,2): # data preprocessing
    if DataPreproc==0:
        TempData=MainNoNanModelSpace
        TotalRegions=346
        TotalSubjects=TempData.shape[2]
    elif DataPreproc==1:
        TempData=GSRNoNanModelSpace
        TotalRegions=346
        TotalSubjects=TempData.shape[2]

    for TempThreshold in [0.4,0.3,0.25,0.2,0.175,0.150,0.125,0.1,0.09,0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]: # FC threshold level

        for BCT_Num in range(0,16): # Graph theory measure
        
            TempResults=np.zeros([TotalSubjects,TotalRegions])
            for SubNum in range(0,TotalSubjects):
                x = bct.threshold_proportional(TempData[:,:,SubNum], TempThreshold, copy=True)
                if BCT_Num==7:
                    s=np.asarray(BCT_models[BCT_Num][1](x,1))
                elif BCT_Num==8:
                    temp_s=np.asarray(BCT_models[BCT_Num][1](x))
                    s=temp_s[0]
                elif BCT_Num==9:
                    temp_s=np.asarray(BCT_models[BCT_Num][1](x))
                    s=temp_s[0]
                elif BCT_Num==10:
                    s=np.asarray(BCT_models[BCT_Num][1](x,KeptYeoIDs))
                elif BCT_Num==11:
                    s=np.asarray(BCT_models[BCT_Num][1](x,KeptYeoIDs))
                elif BCT_Num==12:
                    s=np.asarray(BCT_models[BCT_Num][1](x,0.85))
     
                elif BCT_Num==13:
                    temp_s=np.asarray(BCT_models[BCT_Num][1](x,KeptYeoIDs))
                    s=temp_s[0] 
                elif BCT_Num==14:
                    temp_s=np.asarray(BCT_models[BCT_Num][1](x,KeptYeoIDs))
                    s=temp_s[0] 
                elif BCT_Num==15:
                    temp_s=np.asarray(BCT_models[BCT_Num][1](x))
                    s=temp_s[0]
                else:
                    s=np.asarray(BCT_models[BCT_Num][1](x))
                TempResults[SubNum,:]=s #For each subject for each approach keep the 346 regional values.        

            BCT_Run[count]=BCT_models[BCT_Num][0];
            Sparsities_Run[count]=TempThreshold
            Data_Run[count]=DataPreproc
            GroupSummary[count]='Mean'
            cos_sim=cosine_similarity(TempResults,TempResults) #Build an array of similarities between subjects for each analysis approach 

            if Results.size == 0:
                Results=np.mean(TempResults,axis=0)
                ResultsIndVar=np.asarray(list(cos_sim[np.triu_indices(TotalSubjects,k = 1)])).T #Keep the upper triangle of similarity matrix, reshape to a 1D vector resulting a 2D array of between subject similarities x analysis approach 
            else:
                Results=np.vstack((Results,np.mean(TempResults,axis=0)))
                ResultsIndVar=np.vstack((ResultsIndVar,np.asarray(list(cos_sim[np.triu_indices(TotalSubjects,k = 1)]))))
            count=count+1
            

            
ModelsResults={"Results": Results, "ResultsIndVar": ResultsIndVar, "BCT": BCT_Run,"Sparsities": Sparsities_Run, "Data": Data_Run, "SummaryStat": GroupSummary}
            
pickle.dump( ModelsResults, open(str(output_path / "ModelsResults.p"), "wb" ) )



## Building the low-dimensional space

### LLE, SE, tSNE Analysis

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt

from sklearn import manifold, datasets

from collections import OrderedDict
from functools import partial
from time import time
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle
import numpy as np

ModelResults=pickle.load(open(str(output_path / "ModelsResults.p"), "rb" ) )
Results=ModelResults['ResultsIndVar']
BCT_Run=ModelResults['BCT']
Sparsities_Run=ModelResults['Sparsities']
Data_Run=ModelResults['Data']
GroupSummary=ModelResults['SummaryStat']
Ages=np.asarray(data1['age.main'])

#Scale the data prior to dimensionality reduction
scaler = StandardScaler()
X=scaler.fit_transform(Results.T)

X=X.T

n_neighbors = 20
n_components = 2 #number of components requested. In this case for a 2D space.

fig = plt.figure(figsize=(12, 8))

#Define different dimensionality reduction techniques 
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='dense')

methods = OrderedDict()
methods['LLE'] = LLE(method='standard')
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,n_neighbors=n_neighbors)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca',random_state=0)
#methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=10, random_state=21,metric=True)


markers=["x","s","o","*","D","1","v","p","H","+","|","_","3","^","4","<","X"]
colourmaps=["Oranges","Purples","Greens"]
BCT = np.array(list(BCT_Run.items()))[:,1]
Sparsities = np.array(list(Sparsities_Run.items()))[:,1]
Data=np.array(list(Data_Run.items()))[:,1]

# Reduced dimensions
data_reduced = {}

fig.subplots_adjust(right=0.7)
figDE = plt.figure(constrained_layout=False,figsize=(21,6))
gsDE = figDE.add_gridspec(nrows=6, ncols=21)#, left=0.05, right=0.48, wspace=0.05)

#Perform embedding and plot the results (including info about the approach in the color/intensity and shape).

for i, (label, method) in enumerate(methods.items()):
     
    t0 = time()
    Y = method.fit_transform(X)

    t1 = time()
    # Save the results
    data_reduced[label] = Y
    
    ax = figDE.add_subplot(gsDE[:,i*6+i:(i+1)*6+i])
    for j, d in enumerate(np.unique(Data)):

        BCTTemp=BCT[Data==d]
        SparsitiesTemp=Sparsities[Data==d]
        YTemp=Y[Data==d,:]

        
        for i, c in enumerate(np.unique(BCTTemp)):
            im=ax.scatter(YTemp[:,0][BCTTemp==c],YTemp[:,1][BCTTemp==c],
                          c=SparsitiesTemp[BCTTemp==c]*-0.6, marker=markers[i],
                          cmap=colourmaps[j],s=80)

    ax.set_title("%s " % (label),fontsize=15,fontweight="bold")

    ax.axis('tight')

OrangePatch = mpatches.Patch(color='orange', label='Motion Regression')
PurplePatch = mpatches.Patch(color='purple', label='Global Signal Regression')


Lines={}
for i in range(0,16):
    Lines[i] = mlines.Line2D([], [], color='black', linestyle='None',
                             marker=markers[i],markersize=10, label=np.unique(BCT)[i])


figDE.savefig(str(output_path / 'DifferentEmbeddings.png'),dpi=300)
figDE.savefig(str(output_path / 'DifferentEmbeddings.svg'),format="svg")
figDE.show()


In [ ]:
#Do the same as above but for MDS
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=10, random_state=21,metric=True)
Y = methods['MDS'].fit_transform(X)
data_reduced['MDS'] = Y

markers=["x","s","o","*","D","1","v","p","H","+","|","_","3","^","4","<","X"]
colourmaps=["Oranges","Purples","Greens"]
BCT = np.array(list(BCT_Run.items()))[:,1]
Sparsities = np.array(list(Sparsities_Run.items()))[:,1]
Data=np.array(list(Data_Run.items()))[:,1]

figMDS = plt.figure(constrained_layout=False,figsize=(21,15))
gsMDS = figMDS.add_gridspec(nrows=15, ncols=20)


ax = figMDS.add_subplot(gsMDS[:,0:15])


for j, d in enumerate(np.unique(Data)):

    BCTTemp=BCT[Data==d]
    SparsitiesTemp=Sparsities[Data==d]
    YTemp=Y[Data==d,:]

    for i, c in enumerate(np.unique(BCTTemp)):
        im=ax.scatter(YTemp[:,0][BCTTemp==c],YTemp[:,1][BCTTemp==c],
                      c=SparsitiesTemp[BCTTemp==c]*0.1, marker=markers[i]
                      ,cmap=colourmaps[j],s=150)
        ax.spines['top'].set_linewidth(1.5)
        ax.spines['right'].set_linewidth(1.5)
        ax.spines['bottom'].set_linewidth(1.5)
        ax.spines['left'].set_linewidth(1.5)
        ax.set_xlabel('Dimension 2',fontsize=20,fontweight="bold")
        ax.set_ylabel('Dimension 1',fontsize=20,fontweight="bold")
        ax.tick_params(labelsize=15)


ax.set_title('Multi-dimensional Scaling', fontsize=25,fontweight="bold")


OrangePatch = mpatches.Patch(color='orange', label='Motion Regression')
PurplePatch = mpatches.Patch(color=[85/255, 3/255, 152/255], label='Global Signal Regression')

IntensityPatch1 = mpatches.Patch(color=[0.1, 0.1, 0.1], label='Threshold: 0.4',alpha=1)
IntensityPatch2 = mpatches.Patch(color=[0.1, 0.1, 0.1], label='Threshold: 0.1',alpha=0.4)
IntensityPatch3 = mpatches.Patch(color=[0.1, 0.1, 0.1], label='Threshold: 0.01',alpha=0.1)

BlankLine=mlines.Line2D([], [], linestyle='None')

Lines={}
for i in range(0,16):
    Lines[i] = mlines.Line2D([], [], color='black', linestyle='None',marker=markers[i],markersize=10, label=np.unique(BCT)[i])

figMDS.legend(handles=[OrangePatch, PurplePatch,BlankLine,IntensityPatch1,IntensityPatch2, IntensityPatch3,BlankLine,Lines[0],Lines[1],Lines[2],Lines[3],Lines[4],Lines[5],Lines[6],Lines[7],Lines[8],Lines[9],Lines[10],Lines[11],Lines[12],Lines[13],Lines[14],Lines[15]],fontsize=15,frameon=False,bbox_to_anchor=(1.4, 0.8),bbox_transform=ax.transAxes)#bbox_to_anchor=(1, 1),bbox_transform=f.gcf().transFigure)
#Keep embedding space for subsequent use in active learning below
ModelEmbedding=Y 
figMDS.savefig(str(output_path / 'MDSSpace.png'),dpi=300)
figMDS.savefig(str(output_path /'MDSSpace.svg'),format="svg")


## Analyse the neighbours

In [ ]:
from sklearn.neighbors import kneighbors_graph

def get_dissimilarity_n_neighbours(all_neighbours_orig,
                                   all_neighbours_reduced,
                                   n_neighbours):
    '''
    Calculate the dissimilarity
    
    Parameters
    ----------
    all_neighbours_orig:
    all_neighbours_reduced:

    Returns
    -------
    all_dissimilarity: Dissimilarity scores between the original and reduced
    space
    '''

    all_dissimilarity = []
    for K in n_neighbours:
        # Find the set of different indices
        diff = set(all_neighbours_orig[K]) - set(all_neighbours_reduced[K])
        # Calculate the dissimilarity
        epsilon = len(diff) / len(all_neighbours_orig[K])
        all_dissimilarity.append(epsilon)

    return all_dissimilarity

def get_models_neighbours(n_neigbours, data):
    '''
    Calculate the dissimilarity
    
    Parameters
    ----------
    n_neighbours: number of neighbours to analyse
    data: data (pairwise_subjects, n_analysis)

    Returns
    -------
    all_dissimilarity: Dissimilarity scores between the original and reduced
    space
    '''
    all_adj = np.zeros((len(data), len(data), len(n_neighbours)))
    all_neighbours_orig = []
    
    for idx, n_neighbour in enumerate(n_neighbours):
        adj = kneighbors_graph(data, n_neighbour, mode='distance',
                            metric='euclidean')
        adj_array = adj.toarray()
        all_adj[:, :, idx] = adj_array
        nneighbours_orig = np.nonzero(adj_array)
        nneighbours_orig = [item for item in zip(nneighbours_orig[0],
                                                 nneighbours_orig[1])]
        all_neighbours_orig.append(nneighbours_orig)

    return all_neighbours_orig, all_adj

In [ ]:
N = 544
n_neighbours = range(2, N+1, 10)

neighbours_orig, _ = get_models_neighbours(n_neighbours, X)

In [ ]:
neighbours_tsne, _ = get_models_neighbours(n_neighbours, data_reduced['t-SNE'])
diss_tsne = get_dissimilarity_n_neighbours(neighbours_orig, neighbours_tsne,
                                           n_neighbours)
del neighbours_tsne 

In [ ]:
neighbours_lle, _ = get_models_neighbours(n_neighbours, data_reduced['LLE'])
diss_lle = get_dissimilarity_n_neighbours(neighbours_orig,neighbours_lle)
del neighbours_lle 

In [ ]:
neighbours_se, _ = get_models_neighbours(n_neighbours, data_reduced['SE'])
diss_se = get_dissimilarity_n_neighbours(neighbours_orig,neighbours_se)
del neighbours_se

In [ ]:
neighbours_mds, _ = get_models_neighbours(n_neighbors,data_reduced['MDS'])
diss_mds = get_dissimilarity_n_neighbours(neighbours_orig,neighbours_mds)
del neighbours_mds

In [ ]:
from scipy.stats import hypergeom
# Calculate the null distribution
def expectation(N,K):
    rv = hypergeom(N, K, K)
    x = np.arange(0, K)
    pmf = rv.pmf(x)
    return np.sum(x*pmf)

null_distribution = []
for K in n_neighbours:
    E = expectation(N,K)
    diss = 1 - (E/K)
    null_distribution.append(diss)

In [ ]:
# Get random distribution by brutforce
#print(len(neighbours_orig[2]), len(neighbours_orig[3]))
random_neighbours = 
for K in n_neighbours:
  neighbours_orig[K]


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(n_neighbours, diss_tsne, label='t-SNE', color='#1DACE8')
ax.plot(n_neighbours, diss_lle, label='LLE', color='#E5C4A1')
ax.plot(n_neighbours, diss_se, label='SE', color='#F24D29')
ax.plot(n_neighbours, diss_mds, label='MDS', color='#1C366B')
plt.plot(n_neighbours, null_distribution, label='random', c='grey')
#ax.axvline(x=250, c='grey', linestyle='--', alpha=0.7)
plt.ylim([0,1])
plt.xlim([0,max_neig])
plt.legend(frameon=False)
plt.xlabel('$k$ Nearest Neighbors')
plt.ylabel('Dissimilarity $\epsilon_k$')
plt.savefig(str(output_path / 'dissimilarity_all.svg'))
plt.show()

## Exhaustive Search

Exhaustive search for linearSVR prediction of age, so we know what "ground truth" is.

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from helperfunctions import objectiveFunc,bayesian_optimisation, display_gp_mean_uncertainty
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, RBF, ConstantKernel
from sklearn.neighbors import NearestNeighbors
from sklearn.gaussian_process import GaussianProcessRegressor
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
PredictedAcc=np.zeros((len(Data_Run)))
for i in range(len(Data_Run)):
    #print('Next Iteration:')
    #print(i)
    tempPredAcc=objectiveFunc(i,AgesPrediction,Sparsities_Run,Data_Run,
                              BCT_models,BCT_Run,KeptYeoIDs,MainNoNanPrediction,
                              GSRNoNanPrediction,1)
    #print(tempPredAcc)
    PredictedAcc[i]=tempPredAcc

#Display how predicted accuracy is distributed across the low-dimensional space
plt.scatter(ModelEmbedding[0:PredictedAcc.shape[0],0],ModelEmbedding[0:PredictedAcc.shape[0],1],c=PredictedAcc)
plt.colorbar()

In [ ]:
# Dump accuracies
pickle.dump(PredictedAcc, open(str(output_path / 'predictedAcc.pckl'), 'wb'))

## Active Learning

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from helperfunctions import objectiveFunc,bayesian_optimisation, display_gp_mean_uncertainty
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, RBF, ConstantKernel
from sklearn.neighbors import NearestNeighbors
from sklearn.gaussian_process import GaussianProcessRegressor
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D


#Helper function for calculating posterior predictions only for points in the space where an analysis approach exists 
def posteriorOnlyModels(gp, x_obs, y_obs, z_obs, AllModelEmb):
    xy = (np.array([x_obs.ravel(), y_obs.ravel()])).T
    gp.fit(xy, z_obs)
    mu, std = gp.predict(AllModelEmb, return_std=True)
    return mu, std, gp


#Define the kernel: white noise kernel plus Mattern
kernel = 1.0 * Matern(length_scale=25, length_scale_bounds=(10,80),nu=2.5) \
    + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 0.1))



lb1 = np.min(ModelEmbedding[:, 0])
hb1 = np.max(ModelEmbedding[:, 0])
lb2 = np.min(ModelEmbedding[:, 1])
hb2 = np.max(ModelEmbedding[:, 1])
pbounds = {'b1': (lb1, hb1), 'b2': (lb2, hb2)}



#For finding nearest point in space to next suggested sample from Bayesian optimization
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(ModelEmbedding)

distances, indices = nbrs.kneighbors(ModelEmbedding)

#Acquisition function, in this case upper confidence bound with exploratory kappa vaalue
utility = UtilityFunction(kind="ucb", kappa=10,xi=1e-1)

init_points=10 #Number of burn in random initial samples
n_iter=40 #Number of iterations of Bayesian optimization after burn in

RandomSeed=118

#Initialise optimizer    
optimizer = BayesianOptimization(f=None,
                                pbounds=pbounds,
                                 verbose=4,
                                 random_state=RandomSeed)

optimizer.set_gp_params(kernel=kernel,normalize_y=True,n_restarts_optimizer=10)
np.random.seed(RandomSeed)

# Perform optimization. Given that the space is continuous and the analysis 
# approaches are not, we penalize suggestions that are far from any actual 
# analysis approaches. For these suggestions the registered value is set to the
#  lowest value from the burn in. These points (FailedIters) are only used
# during search but exluded when recalculating the GP regression after search.
FailedIters=bayesian_optimisation(kernel, optimizer, utility, init_points,
                                  n_iter, pbounds, nbrs,RandomSeed,
                                  ModelEmbedding,BCT_models,BCT_Run,
                                  Sparsities_Run,Data_Run,AgesPrediction,
                                  KeptYeoIDs,MainNoNanPrediction,
                                  GSRNoNanPrediction,1,-1)


In [ ]:
#Display the results of the active search and the evolution of the search after 5, 10,20, 30 and 50 iterations.
from scipy.stats import spearmanr
from itertools import product

def _posterior(gp, x_obs, y_obs, z_obs, grid_X):
    xy = (np.array([x_obs.ravel(), y_obs.ravel()])).T
    gp.fit(xy, z_obs)
    mu, std = gp.predict(grid_X.reshape(-1, 2), return_std=True)
    return mu, std, gp

def posteriorOnlyModels(gp, x_obs, y_obs, z_obs, AllModelEmb):
    xy = (np.array([x_obs.ravel(), y_obs.ravel()])).T
    gp.fit(xy, z_obs)
    mu, std = gp.predict(AllModelEmb, return_std=True)
    return mu, std, gp




BadIter=FailedIters
x = np.linspace(pbounds['b1'][0] - 10, pbounds['b1'][1] + 10, 500).reshape(
    -1, 1)
y = np.linspace(pbounds['b2'][0] - 10, pbounds['b2'][1] + 10, 500).reshape(
    -1, 1)
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True,
                              n_restarts_optimizer=10)

x_temp = np.array([[res["params"]["b1"]] for res in optimizer.res])
y_temp = np.array([[res["params"]["b2"]] for res in optimizer.res])
z_temp = np.array([res["target"] for res in optimizer.res])

x_obs=x_temp[BadIter==0]
y_obs=y_temp[BadIter==0]
z_obs=z_temp[BadIter==0]

NumSamplesToInclude=x_obs.shape[0]
x1x2 = np.array(list(product(x, y)))
X0p, X1p = x1x2[:, 0].reshape(500, 500), x1x2[:, 1].reshape(500, 500)

mu, sigma, gp = _posterior(gp, x_obs[0:NumSamplesToInclude], y_obs[0:NumSamplesToInclude], z_obs[0:NumSamplesToInclude], x1x2)

Zmu = np.reshape(mu, (500, 500))
Zsigma = np.reshape(sigma, (500, 500))

conf0 = np.array(mu - 2 * sigma).reshape(500, 500)
conf1 = np.array(mu + 2 * sigma).reshape(500, 500)

X0p, X1p = np.meshgrid(x, y, indexing='ij')

font_dict_title = {'fontsize': 25}
font_dict_label = {'fontsize': 15}
font_dict_label3 = {'fontsize': 15}
print(x_obs.shape)
vmax=Zmu.max()
vmin=Zmu.min()


cm = ['coolwarm', 'seismic']


fig, (ax1,ax2) = plt.subplots(1,2,figsize=(16,8))

ax = ax1
pcm = ax.pcolormesh(X0p, X1p, Zmu,vmax=vmax,vmin=vmin,cmap=cm[0],rasterized=True)  
ax.set_xlim(-50, 50)
ax.set_ylim(-50, 50)
ax.set_aspect('equal', 'box')
ax = ax2
pcm = ax.scatter(ModelEmbedding[0:PredictedAcc.shape[0],0],ModelEmbedding[0:PredictedAcc.shape[0],1],c=PredictedAcc,vmax=vmax,vmin=vmin,cmap=cm[0],rasterized=True)
ax.set_aspect('equal', 'box')

fig.tight_layout()
ax.set_xlim(-50, 50)
ax.set_ylim(-50, 50)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.825, 0.35, 0.02, 0.3])

fig.colorbar(pcm, cax=cbar_ax)


fig.savefig(str(output_path / 'BOptAndTrueK01.png'),dpi=300)
fig.savefig(str(output_path / 'BOptAndTrueK10.svg'),format='svg',dpi=300)



font_dict_title = {'fontsize': 15}
font_dict_label = {'fontsize': 10}
font_dict_label3 = {'fontsize': 10}

fig, axs = plt.subplots(5,3,figsize=(12,18))
count=0
for NumSamplesToInclude in [5,10,20,30,50]:

    x1x2 = np.array(list(product(x, y)))
    X0p, X1p = x1x2[:, 0].reshape(500, 500), x1x2[:, 1].reshape(500, 500)
    mu, sigma, gp = _posterior(gp, x_obs[0:NumSamplesToInclude], y_obs[0:NumSamplesToInclude], z_obs[0:NumSamplesToInclude], x1x2)
    muModEmb,sigmaModEmb,gpModEmb=posteriorOnlyModels(gp, x_obs[0:NumSamplesToInclude], y_obs[0:NumSamplesToInclude], z_obs[0:NumSamplesToInclude],ModelEmbedding)
    Zmu = np.reshape(mu, (500, 500))
    Zsigma = np.reshape(sigma, (500, 500))

    conf0 = np.array(mu - 2 * sigma).reshape(500, 500)
    conf1 = np.array(mu + 2 * sigma).reshape(500, 500)

    X0p, X1p = np.meshgrid(x, y, indexing='ij')

    ax = axs[count,0]

    pcm = ax.pcolormesh(X0p, X1p, Zmu,vmax=vmax,vmin=vmin,cmap=cm[0],rasterized=True)
    ax.set_aspect('equal', 'box')
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)
    ax = axs[count,1]

    pcm = ax.pcolormesh(X0p, X1p, Zsigma,cmap=cm[1],rasterized=True)#,vmax=vmax,vmin=vmin)    
    ax.set_title("Iterations: %i" % (NumSamplesToInclude),fontsize=15,fontweight="bold")
    ax.set_aspect('equal', 'box')
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)

    ax = axs[count,2]
    pcm=ax.scatter(muModEmb[PredictedAcc!=PredictedAcc.min()],PredictedAcc[PredictedAcc!=PredictedAcc.min()],marker='.',c='gray',rasterized=True)
    ax.set_xlim(-0.225, -0.265)
    ax.set_ylim(-0.225, -0.265)
    ax.set_aspect('equal', 'box')

    count=count+1



fig.savefig(str(output_path / 'BOptEvolutionK10.svg'),format='svg',dpi=300)


#Calculate the correlation between the actual and predicted space

print(np.corrcoef(muModEmb,PredictedAcc))
print(spearmanr(muModEmb,PredictedAcc))

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from helperfunctions import objectiveFunc, bayesian_optimisation, display_gp_mean_uncertainty
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, RBF, ConstantKernel
from sklearn.neighbors import NearestNeighbors
from sklearn.gaussian_process import GaussianProcessRegressor
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import SVR
from sklearn.model_selection import permutation_test_score

kernel = 1.0 * Matern(length_scale=25, length_scale_bounds=(10,80),nu=2.5) \
    + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 0.1))

lb1 = np.min(ModelEmbedding[:, 0])
hb1 = np.max(ModelEmbedding[:, 0])
lb2 = np.min(ModelEmbedding[:, 1])
hb2 = np.max(ModelEmbedding[:, 1])
pbounds = {'b1': (lb1, hb1), 'b2': (lb2, hb2)}

BestModelGPSpace=np.zeros(20)
BestModelGPSpaceModIndex=np.zeros(20)
BestModelEmpirical=np.zeros(20)
BestModelEmpiricalModIndex=np.zeros(20)
ModelActualAccuracyCorrelation=np.zeros(20)
CVPValBestModels=np.zeros(20)

for DiffInit in range(0,20):
    optimizer = BayesianOptimization(f=None,
                                     pbounds=pbounds,
                                     verbose=4,
                                     random_state=166+DiffInit)

    optimizer.set_gp_params(kernel=kernel,normalize_y=True,n_restarts_optimizer=10)

    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(ModelEmbedding)

    distances, indices = nbrs.kneighbors(ModelEmbedding)

    utility = UtilityFunction(kind="ucb", kappa=10,xi=1e-1)


    n_iter=10
    init_points=10
    RandomSeed=111+DiffInit
    np.random.seed(RandomSeed)
    FailedIters=bayesian_optimisation(kernel, optimizer, utility, init_points,
                                      n_iter, pbounds, nbrs,RandomSeed,
                                      ModelEmbedding,BCT_models,BCT_Run,
                                      Sparsities_Run,Data_Run,AgesPrediction,
                                      KeptYeoIDs,MainNoNanPrediction,
                                      GSRNoNanPrediction,1,-1)
    
    gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True,
                                  n_restarts_optimizer=10)

    x_temp = np.array([[res["params"]["b1"]] for res in optimizer.res])
    y_temp = np.array([[res["params"]["b2"]] for res in optimizer.res])
    z_temp = np.array([res["target"] for res in optimizer.res])

    x_obs=x_temp[FailedIters==0]
    y_obs=y_temp[FailedIters==0]
    z_obs=z_temp[FailedIters==0]
    
    muModEmb,sigmaModEmb,gpModEmb=posteriorOnlyModels(gp, x_obs, y_obs, z_obs,ModelEmbedding)
    
    BestModelGPSpace[DiffInit]=muModEmb.max()
    BestModelGPSpaceModIndex[DiffInit]=muModEmb.argmax()
    BestModelEmpirical[DiffInit]=z_obs.max()
    Model_coord = np.array([[x_obs[z_obs.argmax()][-1], y_obs[z_obs.argmax()][-1]]])
    BestModelEmpiricalModIndex[DiffInit]=nbrs.kneighbors(Model_coord)[1][0][0]
    ModelActualAccuracyCorrelation[DiffInit]=spearmanr(muModEmb,PredictedAcc)[0]
    
    ClassOrRegress=1
    TempModelNum=muModEmb.argmax()
    Y=AgesPrediction
    CommunityIDs=KeptYeoIDs
    if Data_Run[TempModelNum]==0:
        TempData=MainNoNanPrediction # BUG BUG BUG 
        TotalRegions=346
        TotalSubjects=TempData.shape[2]
    elif Data_Run[TempModelNum]==1:
        TempData=GSRNoNanPrediction
        TotalRegions=346
        TotalSubjects=TempData.shape[2]
    
    
    
    TempThreshold=Sparsities_Run[TempModelNum]
    
    BCT_Num=[i for i, e in enumerate(BCT_models) if e[0] == BCT_Run[TempModelNum]][0]
    
    TempResults=np.zeros([TotalSubjects,TotalRegions])
    for SubNum in range(0,TotalSubjects):
        x = bct.threshold_proportional(TempData[:,:,SubNum], TempThreshold, copy=True)
        if BCT_Num==7:
            s=np.asarray(BCT_models[BCT_Num][1](x,1))
        elif BCT_Num==8:
            temp_s=np.asarray(BCT_models[BCT_Num][1](x))
            s=temp_s[0]
        elif BCT_Num==9:
            temp_s=np.asarray(BCT_models[BCT_Num][1](x))
            s=temp_s[0]
        elif BCT_Num==10:
            s=np.asarray(BCT_models[BCT_Num][1](x,CommunityIDs))
        elif BCT_Num==11:
            s=np.asarray(BCT_models[BCT_Num][1](x,CommunityIDs))
        elif BCT_Num==12:
            s=np.asarray(BCT_models[BCT_Num][1](x,0.85))
                #elif BCT_Num==13:
                #   temp_s=np.asarray(BCT_models[BCT_Num][1](x))
                #   s=temp_s[0]
        elif BCT_Num==13:
                #temp_s=np.asarray(BCT_models[BCT_Num][1](x,KeptYeoIDs,'degree'))
            temp_s=np.asarray(BCT_models[BCT_Num][1](x,CommunityIDs))
            s=temp_s[0] 
        elif BCT_Num==14:
            temp_s=np.asarray(BCT_models[BCT_Num][1](x,CommunityIDs))
            s=temp_s[0] 
        elif BCT_Num==15:
            temp_s=np.asarray(BCT_models[BCT_Num][1](x))
            s=temp_s[0]
        else:
            s=np.asarray(BCT_models[BCT_Num][1](x))

        TempResults[SubNum,:]=s 
    scaler = StandardScaler()
    TempResults=scaler.fit_transform(TempResults)
  
    model = SVR(C=1.0, epsilon=0.2)
    rs = np.random.RandomState(100)
    TempScore=permutation_test_score(model, TempResults, AgesPrediction.ravel(),
                                     groups=None, cv=None, n_permutations=5000, 
                                     n_jobs=None, random_state=5, verbose=0,
                                     scoring="neg_mean_absolute_error")
    CVPValBestModels[DiffInit]=TempScore[2]
    

In [ ]:
#TEMP
results = {}
results['ModelEmbedding'] = ModelEmbedding
results['BestModelGPSpaceModIndex'] = BestModelGPSpaceModIndex
results['BestModelEmpiricalModIndex'] = BestModelEmpiricalModIndex
results['BestModelEmpirical'] = BestModelEmpirical
results['ModelActualAccuracyCorrelation'] = ModelActualAccuracyCorrelation
results['TempResults'] = TempResults
pickle.dump(results, open(str(output_path / 'results.pckl'), 'wb'))

In [ ]:
# displaying results of 20 iterations

fig8 = plt.figure(constrained_layout=False,figsize=(18,6))
gs1 = fig8.add_gridspec(nrows=6, ncols=18)
ax1 = fig8.add_subplot(gs1[:,0:6])
ax1.set_title('Optima GP regression: 20 iterations',fontsize=15,fontweight="bold")
ax1.scatter(ModelEmbedding[0:PredictedAcc.shape[0],0],
            ModelEmbedding[0:PredictedAcc.shape[0],1],
            c=PredictedAcc,vmax=vmax,vmin=vmin,cmap='coolwarm',alpha=0.2,s=120)
ax1.scatter(ModelEmbedding[BestModelGPSpaceModIndex.astype(int)][:,0],
            ModelEmbedding[BestModelGPSpaceModIndex.astype(int)][:,1],s=120,c='black')

ax1.set_xlim(-50, 50)
ax1.set_ylim(-50, 50)

ax2 = fig8.add_subplot(gs1[:,7:13])
ax2.set_title('Empirical optima: 20 iterations',fontsize=15,fontweight="bold")
ax2.scatter(ModelEmbedding[0:PredictedAcc.shape[0],0],
            ModelEmbedding[0:PredictedAcc.shape[0],1],
            c=PredictedAcc,vmax=vmax,vmin=vmin,cmap='coolwarm',s=120,alpha=0.2)
ax2.scatter(ModelEmbedding[BestModelEmpiricalModIndex.astype(int)][:,0],
            ModelEmbedding[BestModelEmpiricalModIndex.astype(int)][:,1],c='black',s=120)

ax2.set_xlim(-50, 50)
ax2.set_ylim(-50, 50)

ax3 = fig8.add_subplot(gs1[:,14:16])
ax3.violinplot([PredictedAcc,BestModelEmpirical])
ax3.set_xticks([1, 2])
ax3.set_xticklabels(['Accuracy \n of all points', 'Accuracy\n of optima'],fontsize=9)

ax4 = fig8.add_subplot(gs1[:,17:18])
ax4.violinplot([ModelActualAccuracyCorrelation])
ax4.set_xticks([1])
ax4.set_xticklabels(['Correlation: \n est vs emp '],fontsize=9)

gs1
fig8.savefig(str(output_path / 'BOpt20Repeats.png'),dpi=300) 
fig8.savefig(str(output_path / 'BOpt20Repeats.svg'),format="svg") 